In [5]:
import os
import base64
import json
import re
import requests
from dotenv import load_dotenv
from tqdm import tqdm
from langchain_community.document_loaders import PyPDFLoader
from pdf2image import convert_from_path
import openai

# Cargar las variables de entorno desde el archivo .env
load_dotenv(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\.env')

# Obtener la clave de API desde las variables de entorno
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key  # Configurar la clave de API para OpenAI

# Función para codificar la imagen en base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Función para convertir PDF en imágenes (una por página)
def pdf_to_images(pdf_path, output_dir):
    images = convert_from_path(pdf_path)
    image_paths = []
    for i, image in enumerate(images):
        image_path = os.path.join(output_dir, f"page_{i+1}.jpg")
        image.save(image_path, 'JPEG')
        image_paths.append(image_path)
    return image_paths

def create_image_messages(image_path, report_id, page_number):
    # Crear el mensaje inicial
    messages = [{
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": (
                    "I have exported the following dashboard as image. The image corresponds to a dashboard with key metrics **KPIs**, **visualizations**, and **table values**. "
                    "Your task is to identify **ALL** the **KPIs**, **visualizations**, and **table values** shown in each image, providing their exact titles, values, and any other relevant information visible. "
                    "Additionally, provide a description for each element. Explain what each KPI represents, and for the visualizations, describe with inner details what is being shown (e.g., trends, comparisons, etc.). "
                    "As an expert image analyzing tool you have to capture all the information contained in the image and insert it in the response. "
                    "Return **only** a JSON object without any text before or after the JSON, and without any Markdown or special characters. The JSON should follow this exact structure: "
                    "{"
                    "  \"report_id\": \"" + report_id + "\","
                    "  \"date\": \"2024-09-23\","
                    "  \"page\": " + str(page_number) + ","
                    "  \"detected_elements\": {"
                    "    \"KPIs\": ["
                    "      {"
                    "        \"kpi_title\": \"<Exact KPI title>\","
                    "        \"kpi_value\": \"<KPI value>\","
                    "        \"kpi_reference\": \"<KPI reference or baseline>\","
                    "        \"kpi_description\": \"<Description explaining the significance of the KPI including the current value of the KPI >\""
                    "      }"
                    "      ..."
                    "    ],"
                    "    \"charts\": ["
                    "      {"
                    "        \"visualization_title\": \"<Exact visualization title>\","
                    "        \"visualization_type\": \"<Bar Chart, Line Chart, Pie Chart, etc.>\","
                    "        \"metrics_displayed\": [\"<List of metrics in the chart>\"],"
                    "        \"values\": [\"<**All** values from the visualization>\"],"
                    "        \"visualization_description\": \"<Explanation of what the chart is showing with the relevant information (example of relevant info: cost increment trend is shown during year2019, foreigners consum more than the rest >\""
                    "      }"
                    "      ..."
                    "    ],"
                    "    \"tables\": ["
                    "      {"
                    "        \"table_title\": \"<Exact table title>\","
                    "        \"columns\": [\"<List of **all** columns>\"],"
                    "        \"rows\": ["
                    "          [\"<Row 1 values>\"],"
                    "          [\"<Row 2 values>\"],"
                    "          ..."
                    "        ],"
                    "        \"table_description\": \"<Explanation of what the table is showing with the relevant information (example of relevant info: cost increment trend is shown during year2019, foreigners consum more than the rest>\""
                    "      }"
                    "      ..."
                    "    ]"
                    "  }"
                    "}"
                )
            }
        ]
    }]

    # Codificar la imagen en base64 y añadirla al mensaje
    base64_image = encode_image(image_path)
    messages[0]["content"].append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}",
            "detail": "high"
        }
    })

    return messages


def process_image(image_path, report_id, page_number):
    """Procesa las imágenes y devuelve la respuesta de la API."""
    messages = create_image_messages(image_path, report_id, page_number)
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o-mini",
        "messages": messages,
        "max_tokens": 15000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

def process_text_with_openai(page_content, report_id, page_number, processed_content=None):
    """Procesa el contenido de una página en formato de texto."""
    
    # Crea el contenido del mensaje incluyendo el contenido de la página y los resultados procesados
    processed_content_text = processed_content if processed_content else "No processed content available."
    
    messages = [{
        "role": "user",
        "content": (
            f"I have exported the following dashboard as PDF corresponding to report '{report_id}' on page {page_number}. "
            f"Here is the content in plain text extracted from the pdf: {page_content}\n"
            f"Here are the previously processed details from pdf converted to JSON throught chatgpt4o mini image processor: {processed_content_text}\n"
            "As an advanced analyze tool, you are going to carry out a second processing step to improve the first result"
            "Your task is to identify all the missing values and complete the JSON file provided by the image processor."
            "You also have to correct any incorrect data or description"
            "Ensure to maintain the structure received from the image processor and fill in the tables, KPI titles, values, visualization titles, displayed metrics, and descriptions missed in the first processing."
            "Return **only** a JSON object without any text before or after the JSON, and without any Markdown or special characters. The JSON should follow this exact structure: "
                        "{"
                        "  \"report_id\": \"" + report_id + "\","
                        "  \"date\": \"2024-09-23\","
                        "  \"page\": " + str(page_number) + ","
                        "  \"detected_elements\": {"
                        "    \"KPIs\": ["
                        "      {"
                        "        \"kpi_title\": \"<Exact KPI title>\","
                        " ... "
        )
    }]
    
    client = openai.OpenAI()

    try:
        # Llamar a la API
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=15000,
            temperature=0.3,
        )

        response_content = completion.choices[0].message.content.strip()
        return response_content
    
    except Exception as e:
        print(f"Error al llamar a la API de OpenAI: {e}")
        return None


def pdf_to_text(pdf_path):
    """Convierte todas las páginas de un PDF a texto y devuelve una lista de los contenidos por página."""
    loader = PyPDFLoader(pdf_path, extraction_mode="layout")
    pages = loader.lazy_load()

    pdf_text_contents = []
    for idx, doc in enumerate(pages):
        pdf_text_contents.append(doc.page_content)  # Almacena el contenido de cada página como un string

    return pdf_text_contents
    

def main(pdf_path, output_dir):
    """Función principal que coordina el procesamiento de imágenes y texto del PDF."""
    report_id = os.path.basename(pdf_path).replace('.pdf', '')  # Nombre del archivo sin extensión

    # Convertir el PDF a imágenes
    image_paths = pdf_to_images(pdf_path, output_dir)

    # # Convertir todo el PDF a texto
    pdf_text_contents = pdf_to_text(pdf_path)

    # Procesar cada imagen y su correspondiente texto del PDF
    for idx, image_path in enumerate(image_paths):
        # Procesar la imagen
        image_response = process_image(image_path, report_id, idx + 1)

        # Extraer solo el contenido
        content = image_response.get('choices', [{}])[0].get('message', {}).get('content', "")

        # Preparar un diccionario para almacenar los resultados de la página
        page_result = content
        # page_result_path = os.path.join(output_dir, f"{report_id}_page_{idx + 1}_result_v2.json")
        # with open(page_result_path, "w", encoding="utf-8") as jsonfile:
        #     json.dump(page_result, jsonfile, indent=4)

        ###### # # Verificar si el contenido contiene "warning"
        ###### # if content and "warning" in content.lower():
        ######     # Procesar el texto de la página correspondiente del PDF

        page_content = pdf_text_contents[idx]
        text_response = process_text_with_openai(page_content, report_id, idx + 1, content)
        # Guardar el resultado de la página en un archivo JSON
        page_result_path = os.path.join(output_dir, f"{report_id}_page_{idx + 1}_result_v2_corrected.json")
        with open(page_result_path, "w", encoding="utf-8") as jsonfile:
            json.dump(text_response, jsonfile, indent=4)

        print(f"Resultados de la página {idx + 1} guardados en {page_result_path}")



# Ruta al PDF y directorio de salida para las imágenes
pdf_path = r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_PDFs\Rx_Sales_Report.pdf'
output_dir = r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_json_reports'

# Ejecutar la función principal
main(pdf_path, output_dir)


Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.


Resultados de la página 1 guardados en C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_json_reports\Rx_Sales_Report_page_1_result_v2_corrected.json
Resultados de la página 2 guardados en C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_json_reports\Rx_Sales_Report_page_2_result_v2_corrected.json
Resultados de la página 3 guardados en C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_json_reports\Rx_Sales_Report_page_3_result_v2_corrected.json
Resultados de la página 4 guardados en C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_json_reports\Rx_Sales_Report_page_4_result_v2_corrected.json
Resultados de la página 5 guardados en C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistan

In [6]:
import os
import base64
import json
import re
import requests
from dotenv import load_dotenv
from tqdm import tqdm
import openai

# Cargar las variables de entorno desde el archivo .env
load_dotenv(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\.env')

# Obtener la clave de API desde las variables de entorno
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key  # Configurar la clave de API para OpenAI
client = openai.Client()

def load_json_from_file(file_path):
    """Carga un archivo JSON desde la ruta especificada y devuelve el contenido."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def process_json_with_openai(directory):
    """Carga y procesa todos los archivos JSON en un directorio usando la API de OpenAI."""
    combined_json_str = []  # Lista para almacenar el contenido combinado

    # Recorrer todos los archivos en el directorio
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            # Buscar el número de página usando una expresión regular
            match = re.search(r'page_(\d+)', filename)
            if match:
                page_number = int(match.group(1))
                file_path = os.path.join(directory, filename)
                try:
                    json_data = load_json_from_file(file_path)
                    combined_json_str.append((page_number, json_data))
                except json.JSONDecodeError as e:
                    print(f"Error al leer el archivo {filename}: {e}")

    # Ordenar la lista de JSON por número de página
    combined_json_str.sort(key=lambda x: x[0])
    combined_json_str = [data for _, data in combined_json_str]

    # Generar el resumen general
    messages = [{
        "role": "user",
        "content": (
            "Los siguientes JSON representan un reporte de Power BI, donde cada JSON contiene información detallada sobre un cuadro de mando:\n\n```json\n"
            f"{combined_json_str}\n"
            "Por favor, genera un resumen descriptivo completo que describa y explique detalladamente el contenido del reporte completo de PowerBi.\n"
            "- El informe tiene que tener como título resumen general del 'report_id'.\n"
            "- El informe debe tener una introducción que mencione el 'report_id', la fecha del reporte y el número total de páginas.\n"
            "- El informe tiene que proporcionar un un resumen que sea capaz de recoger el contexto general del reporte, sin necesidad de ordenarlo por páginas o cuadros de mando.\n"
            "- Asegúrate de identificar y describir cualquier variación significativa y las principales conclusiones del análisis de los KPIs y gráficos.\n"
            "- En caso de que no se encuentren elementos específicos, no hace falta mencionarlos.\n"
            "- Devuelve **únicamente** un objeto JSON sin texto antes o después del JSON, y sin ningún carácter de Markdown o especial. El JSON debe seguir esta estructura exacta:\n"
            "{\n"
            "    \"contenido\": \"\",\n"
            "    \"metadata\": {\n"
            "        \"Title\": \"Resumen general 'Report_id'\",\n"
            "        \"Report_Id\": \"id del informe\",\n"
            "        \"insertion_year\": \"año de inserción\",\n"
            "        \"insertion_month\": \"mes de inserción\",\n"
            "        \"insertion_day\": \"día de inserción\"\n"
            "    }\n"
            "}\n"
        )
    }]

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=15000,
            temperature=0.4,
        )

        response_content = completion.choices[0].message.content.strip()
        return response_content
    
    except Exception as e:
        print(f"Error al llamar a la API de OpenAI: {e}")
        return None

def process_json_page_with_openai(json_data, page_number, report_id):
    """Procesa un JSON específico usando la API de OpenAI para generar un resumen de una página."""
    messages = [{
        "role": "user",
        "content": (
            "El siguiente JSON representa una página de un reporte completo de Power BI, donde cada JSON contiene información detallada sobre un cuadro de mando:\n\n```json\n"
            f"{json_data}\n"
            "Quiero que generes un resumen descriptivo de la página del informe que estará compuesta por el contenido de un JSON correspondiente a un cuadro de mando y debe incluir lo siguiente en formato de párrafos sin estructurar:\n"
            "- Una breve introducción al principio de cada sección que resuma el contenido de la página o cuadro de mando en cuestión.\n"
            "- Una descripción detallada de los KPIs detectados, especificando el título, valor, referencia y su descripción en el contexto del reporte. Si existe alguna variación o comparación asegúrate de destacarla.\n"
            "- Una descripción exhaustiva de los gráficos, especificando el título, tipo de gráfico, métricas mostradas y valores, describiendo las distribuciones y las tendencias observadas en el gráfico. Asegúrate de que las distribuciones y tendencias descritas se expliquen en base en los datos mostrados en la tabla.\n"
            "- En caso de que no se encuentren elementos, no hace falta mencionar nada sobre ellos.\n"
            "- Devuelve **únicamente** un objeto JSON sin texto antes o después del JSON, y sin ningún carácter de Markdown o especial. El JSON debe seguir esta estructura exacta:\n"
            "{\n"
            "    \"contenido\": \"\",\n"
            "    \"metadata\": {\n"
            "        \"Title\": \"Resumen Cuadro de Mando Página {page_number} del {report_id}\",\n"
            "        \"Report_Id\": \"id del informe\",\n"
            "        \"insertion_year\": \"año de inserción\",\n"
            "        \"insertion_month\": \"mes de inserción\",\n"
            "        \"insertion_day\": \"día de inserción\",\n"
            "        \"page\": {page_number},\n"
            "        \"elements\": {\n"
            "            \"KPI\": \"<Título exacto del KPI>\",\n"
            "            \"Table\": \"<Título exacto de la Tabla>\",\n"
            "            \"Chart\": \"<Título exacto del Gráfico>\",\n"
            "            \"Other\": \"<Otros elementos identificados, si los hay>\"\n"
            "        }\n"
            "    }\n"
            "}\n"
        )
    }]

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=15000,
            temperature=0.4,
        )

        response_content = completion.choices[0].message.content.strip()
        return response_content
    
    except Exception as e:
        print(f"Error al llamar a la API de OpenAI: {e}")
        return None

# Ruta al PDF y directorio de salida para las imágenes
pdf_path = r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_PDFs\Rx_Sales_Report.pdf'

# Procesar el JSON con la función que has definido previamente
response = process_json_with_openai(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_json_reports')

# Definir el nombre del informe como una cadena
nombre = os.path.basename(pdf_path).replace('.pdf', '_informe_corrected')

# Guardar el contenido del resumen general en un archivo JSON
output_file_path = os.path.join(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_text_reports', f'{nombre}.json')

# Escribir el contenido del resumen general en el archivo
with open(output_file_path, 'w', encoding='utf-8') as f:
    f.write(response)

# Procesar cada JSON uno por uno
for filename in os.listdir(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_json_reports'):
    if filename.endswith('.json'):
        file_path = os.path.join(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_json_reports', filename)
        page_number = int(re.search(r'page_(\d+)', filename).group(1))

        # Cargar el JSON específico
        json_data = load_json_from_file(file_path)

        # Procesar el JSON para generar un resumen de la página
        page_response = process_json_page_with_openai(json_data, page_number, nombre)

        # Guardar el contenido de la página en un archivo JSON
        output_page_file_path = os.path.join(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Assistant_chatbot\Data_Structuring\sample_text_reports', f'{nombre}_page_{page_number}.json')
        with open(output_page_file_path, 'w', encoding='utf-8') as f:
            f.write(page_response)
